In [ ]:
#importing the clean dataset:
df = pd.read_csv(r"C:\Users\DELL\Desktop\ML training.csv")

In [ ]:
#setting the modeling pipeline
###data preprocessing with pipline.
### we have one nominal feature- gender and two ordinal features- HTN and DM or IFG
### numerical features will be mean-centered and scaled.
### lastly, iterative imputer will be applied.

numerical_ix = df_clean.select_dtypes(include=['float64']).columns
ordinal_ix = df_clean.select_dtypes(include=['int32']).columns
nominal_ix = df_clean.select_dtypes(include=['object']).columns

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,StandardScaler, PowerTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=0)
numeric_transformer = Pipeline(steps=[('standard&scale',StandardScaler()),("yeo-johnson", PowerTransformer(method='yeo-johnson', standardize=False))])
ordinal_transformer = Pipeline(steps=[('ordinal',  OrdinalEncoder())])
nominal_transformer = Pipeline(steps=[('nominal', OneHotEncoder())])

col_transform = ColumnTransformer(transformers=[("numeric", numeric_transformer, numerical_ix),
        ("nominal", nominal_transformer, nominal_ix),("ordinal",ordinal_transformer,ordinal_ix)],
                                  verbose_feature_names_out=False)

In [ ]:
# setting the LASSO logistic regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(penalty="l1",solver='liblinear',max_iter=1000)

pipe_LR = Pipeline(steps = [('transormations', col_transform), ('imputation', imputer),
                         ('classifier',classifier)])

#setting the dictionary for penalization tunning parameter
para_grid_lasso = {"classifier__C":np.logspace(-4,2,50)}


In [ ]:
# fitting LASSO Logistic regression
from sklearn.model_selection import GridSearchCV

tunning_c = GridSearchCV(pipe_LR, para_grid_lasso,scoring='roc_auc',return_train_score=True)

tunning_c.fit(df_clean,y_tt)

In [ ]:
# setting the SVM 
rom sklearn import svm
classifier_SVM = svm.SVC(tol=0.01,probability=True)

SVM_pipe = Pipeline(steps = [('transormations', col_transform), ('imputation', imputer),
                         ('classifier',classifier_SVM)])

#setting the dictionary for tunning parameters
SVM_grid = SVM_grid = [{'classifier__kernel':['linear'],'classifier__C': np.logspace(-4,1,10)},
              {'classifier__kernel':['poly'],'classifier__C': np.logspace(-4,1,10),'classifier__degree':np.arange(2,4)}]


In [ ]:
# fitting SVM
SVM_tune = GridSearchCV(SVM_pipe, param_grid=SVM_grid, scoring='roc_auc',return_train_score=True,n_jobs=-1,verbose=3)

SVM_tune.fit(df_clean,y_tt)

In [ ]:
# setting the random forest
from sklearn.ensemble import RandomForestClassifier
RF_classifier = RandomForestClassifier(n_estimators=500)
pipe_RF = Pipeline(steps = [('transormations', col_transform),('imputation', imputer),('classifier',RF_classifier)])

#setting the dictionary for tunning parameters
RF_grid = RF_grid = [{'classifier__max_depth':[4,5,6],'classifier__min_samples_split': [18,20,25],
                      'classifier__max_features': [0.1,0.2],'classifier__max_samples':[0.5,0.6],'classifier__min_impurity_decrease':[0.001,0.005,0.01]}]


In [ ]:
# fitting random forest
RF_tune_2 = GridSearchCV(pipe_RF, param_grid=RF_grid, scoring='roc_auc',return_train_score=True,n_jobs=-1,verbose=3)

RF_tune_2.fit(df_clean,y_tt)

In [ ]:
# setting the neural network
from sklearn.neural_network import MLPClassifier

pipe_nn= Pipeline(steps = [('transormations', col_transform),('imputation', imputer),('classifier',deep_layer_nn)])

#setting the network architeture
deep_layer_nn = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(5,3),max_iter=15000,random_state=1)

#setting the dictionary for tunning parameter L2
nn_grid = [{'classifier__alpha':[8,9,10,11,12,13,14]}] 

nn_tune = GridSearchCV(pipe_nn, param_grid=nn_grid, cv=5, scoring='roc_auc',return_train_score=True,n_jobs=-1,verbose=3)

nn_tune.fit(df_clean,y_tt)


In [ ]:
# fitting ANN
nn_tune = GridSearchCV(pipe_nn, param_grid=nn_grid, cv=5, scoring='roc_auc',return_train_score=True,n_jobs=-1,verbose=3)

nn_tune.fit(df_clean,y_tt)

In [ ]:
## setting the XGBoost:
from xgboost import XGBClassifier


xg_classifier = XGBClassifier(objective='binary:logistic',eval_metric='auc')

pipe_XG = Pipeline(steps = [('transormations', col_transform),('imputation', imputer),('classifier',xg_classifier)])

#setting the dictionary for tunning parameters
xg_grid = {'classifier__max_depth':[2,3,4],'classifier__n_estimators':[700,750,800,850,950],
           'classifier__eta':[0.15,0.1,0.01],
           'classifier__colsample_bytree':[0.4,0.5],'classifier__subsample':[0.4,0.5],'classifier__alpha':[7],
           'classifier__lambda':[7]}



In [ ]:
# fitting XGBoost
xg_tune =  GridSearchCV(pipe_XG, param_grid=xg_grid, scoring='roc_auc',return_train_score=True,n_jobs=-1,verbose=3)

xg_tune.fit(df_clean,y_tt)

In [ ]:
#Display ROC curves for each model
#In the same loop take the concatenate dataset of validation folds with FIB-4 NFS and XGBoost predictions to calculate sample size
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve,det_curve
from sklearn.model_selection import StratifiedKFold,
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
label_encoder = LabelEncoder()

# initializing XGBoost with the best parameters 
xg_classifier = XGBClassifier(alpha= xg_tune.best_params_['classifier__alpha'],colsample_bytree= xg_tune.best_params_['classifier__colsample_bytree'],
                              eta= xg_tune.best_params_['classifier__eta'],reg_lambda= xg_tune.best_params_['classifier__lambda'],
                              max_depth=xg_tune.best_params_['classifier__max_depth'],n_estimators=xg_tune.best_params_['classifier__n_estimators'],
                              subsample =xg_tune.best_params_['classifier__subsample'], objective='binary:logistic',eval_metric='auc')

pipe_XG = Pipeline(steps = [('transormations', col_transform),('imputation', imputer),('classifier',xg_classifier)])

sample_size_cal = list()
target_sample_cal = list()
FIB = list()
NFS = list()

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)
skf.get_n_splits(df_clean, y)

fig, ax = plt.subplots(figsize=(6, 6))

fpr=[]
tpr=[]
thresholds=[]
threshold=np.zeros(5)
for i, (train_index, test_index) in enumerate(skf.split(df_clean, y)):
    x_train, x_test = df_clean.iloc[train_index,:], df_clean.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    label_encoder = label_encoder.fit(y_train)
    label_encoded_y_train = label_encoder.transform(y_train)
    label_encoded_y_test = label_encoder.transform(y_test)
    pipe_XG.fit(x_train,label_encoded_y_train)
    sample_size_cal.append(pipe_XG.predict_proba(x_test)[:,1])
    FIB.append(df_clean.iloc[test_index,:].apply(lambda x: (x['age'] * x['AST'])/(x['PLT']*x['ALT']**0.5), axis=1))
    NFS.append(df_clean.iloc[test_index,:].apply(lambda x: (-(1.675) + 0.037 *x['age'] + 0.094 * x['BMI'] + 1.13 * x['DM or IFG'] + 0.99 *x['AST']/x['ALT'] - 0.013 *x['PLT'] - 0.66 *x['albumin']), axis=1))
    target_sample_cal.append(label_encoded_y_test)
    viz = RocCurveDisplay.from_estimator(pipe_XG,x_test,label_encoded_y_test,name=f"ROC fold {i+1}",alpha=0.5,lw=3,ax=ax)

ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    #title="ROC curves for 5 folds",
)
ax.axis("square")
ax.legend(loc="lower right")
plt.show()

In [ ]:
#exporting the evaluation folds to R for sample size calculation

sample_pred=np.concatenate((sample_size_cal[0],sample_size_cal[1],sample_size_cal[2],sample_size_cal[3],sample_size_cal[4]))


sample_target=np.concatenate((target_sample_cal[0],target_sample_cal[1],target_sample_cal[2],target_sample_cal[3],target_sample_cal[4]))

sample_FIB=np.concatenate((FIB[0],FIB[1],FIB[2],FIB[3],FIB[4]))

sample_NFS=np.concatenate((NFS[0],NFS[1],NFS[2],NFS[3],NFS[4]))

data_4_export = {'prediction':sample_pred,'label':sample_target,'NFS':sample_NFS,'FIB':sample_FIB}
data_4_export = pd.DataFrame(data_4_export)

#data_4_export.to_csv('sample size.csv')

In [ ]:
#detection error curves of folds and choosing a threshold subjected to resampling for validation
from sklearn.metrics import DetCurveDisplay,roc_curve
fig, ax = plt.subplots(figsize=(6, 6))
fpr=[]
tpr=[]
thresholds=[]
threshold_xg_90=np.zeros(5)
sensitivity = np.zeros(5)
specificity = np.zeros(5)
for i, (train_index, test_index) in enumerate(skf.split(df_clean, pd.Series(y_tt))):
    x_train, x_test = df_clean.iloc[train_index,:], df_clean.iloc[test_index,:]
    y_train, y_test = pd.Series(y_tt).iloc[train_index], pd.Series(y_tt).iloc[test_index]
    fited=pipe_XG.fit(x_train,y_train)
    viz =  DetCurveDisplay.from_estimator(fited,x_test,y_test,name=f"DET fold {i+1}",alpha=0.5,lw=3,ax=ax)#,pos_label='advanced')
    yhat = fited.predict_proba(x_test)
    fpr, fnr, thresholds= det_curve(y_test, yhat[:, 1],pos_label=1)
    fpr2, tpr, thresholds2 = roc_curve(y_test, yhat[:, 1],pos_label=1)
    print(fnr[np.absolute(fnr-0.1).argmin()])
    print(fpr[np.absolute(fnr-0.1).argmin()])
    print(thresholds[np.absolute(fnr-0.1).argmin()])
    print(threshold_xg_90[i])
    print(tpr[np.absolute(tpr-0.9).argmin()])
    sensitivity[i] = tpr[np.absolute(tpr-0.9).argmin()]
    print(1-fpr2[np.absolute(tpr-0.9).argmin()])
    specificity[i] = 1-fpr2[np.absolute(tpr-0.9).argmin()]
    print(thresholds2[np.absolute(tpr-0.9).argmin()])
    threshold_xg_90[i] = thresholds2[np.absolute(tpr-0.9).argmin()]

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    #title="DET curves for 5 folds",
)
ax.axis("square")
ax.legend(loc="upper right")
plt.show()

In [ ]:
# the final threshold is the mean
threshold_xg_90.mean()

In [ ]:
#importing the test data
df_test = pd.read_csv(r"C:\Users\DELL\Desktop\ML test.csv")
y_test = df_test['Fibrosis']
y_testtt = np.zeros(540)
y_testtt[y_test=='advanced']=1
y_testtt[y_test=='non advanced']=0

In [ ]:
# one-way partial dependence-
from sklearn.inspection import PartialDependenceDisplay,partial_dependence
#choose your feature
pdp_HbA1C = partial_dependence(xg_tune.best_estimator_,df_test,features = 'HbA1C',kind="average")

#visualize
plt.plot(pdp_HbA1C['values'][0], (pdp_HbA1C.average[0]), label = "HbA1C")
plt.ylabel('partial dependence')
plt.legend(loc= "lower right")
plt.show()


In [ ]:
# two-way partial dependence
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d  # noqa: F401
#choose your features 
pdp_HbA1C = partial_dependence(xg_tune.best_estimator_,df_test,features = 'HbA1C',kind="average")

fig, ax = plt.subplots(figsize=(25, 25),subplot_kw={"projection": "3d"})

XX, YY = np.meshgrid(pdp_HbA1C_PLT["values"][0],pdp_HbA1C_PLT["values"][1])
Z = pdp_HbA1C_PLT.average[0].T
surf = ax.plot_surface(XX, YY, Z, rstride=5, cstride=5, cmap='Greys', edgecolor="k")
ax.view_init(elev=20, azim=220)# don't touch 20,220
ax.set_ylabel('Platelets (K/microL)',fontsize=30)
ax.xaxis.labelpad = 15
ax.set_xlabel('HbA1C (%)',fontsize=30)
ax.yaxis.labelpad = 17
ax.set_zlabel('Probability of advanced fibrosis',fontsize=30)
ax.zaxis.labelpad = 17
for label in (ax.get_xticklabels() + ax.get_yticklabels()+ax.get_zticklabels()):label.set_fontsize(30)


ax.view_init(elev=20, azim=120)

plt.show()

In [ ]:
#feature importance derived from training via XGBoost algorithm
importance = xg_tune.best_estimator_[2].feature_importances_
indices = np.argsort(importance)

fig, ax = plt.subplots(figsize=(12, 12))
ax.barh(range(len(importance)),importance[indices],color='grey')
ax.set_yticks(range(len(importance)))
_=ax.set_yticklabels(['Sex: Male','Triglycerides','Sex:Female','Cholesterol','BMI','ALT','Diabetes or IFG','ALP','LDL-c','HDL-c', 'Albumin', 'Hypertension', 'Age', 'Platelets', 'GGT', 'AST', 'HbA1c'])
_=ax.set_title('Feature importance via XGboost')
plt.show()

In [ ]:
# orgnaize feature importance via XGBoost on train and test and logistic regression on test
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(class_weight='balanced',y=y_testtt)

from sklearn.inspection import permutation_importance
premutation_XG = permutation_importance(
    xg_tune.best_estimator_[2], xg_tune.best_estimator_[0:2].transform(df_test),y_testtt, n_repeats=100, random_state=22,sample_weight=sample_weights)

sorted_importances_idx = premutation_XG.importances_mean.argsort()
premutation_XG_df = pd.DataFrame(
    premutation_XG.importances[sorted_importances_idx].T,
    columns=pipe_XG[:-1].get_feature_names_out()[sorted_importances_idx])

premutation_LR = permutation_importance(
    tunning_c.best_estimator_[2], tunning_c.best_estimator_[0:2].transform(df_test), y_testtt, n_repeats=100, random_state=2,sample_weight=sample_weights)

sorted_importances_idx = premutation_LR.importances_mean.argsort()

premutation_LR_df = pd.DataFrame(
    premutation_LR.importances[sorted_importances_idx].T,
    columns=pipe_LR[:-1].get_feature_names_out()[sorted_importances_idx])

from scipy.stats import shapiro,ttest_ind,mannwhitneyu
results_dic = {}
for column in pipe_XG[:-1].get_feature_names_out():
    results_dic[column] = []
    if(shapiro(premutation_LR_df[column])[1]<0.05 and shapiro(premutation_XG_df[column])[1]<0.05):
        results_dic[column]= [ttest_ind(premutation_LR_df[column], premutation_XG_df[column])[1],
                              'T test',premutation_XG_df.columns.get_loc(column),
                             premutation_XG_df[column].mean(),
                              premutation_LR_df.columns.get_loc(column),premutation_LR_df[column].mean()]
    else:
        results_dic[column]= [mannwhitneyu(premutation_LR_df[column], premutation_XG_df[column])[1],
                              'mann-whitney',premutation_XG_df.columns.get_loc(column),
                              premutation_XG_df[column].mean(),
                              premutation_LR_df.columns.get_loc(column),premutation_LR_df[column].mean()]
        


In [ ]:
# The table
results_df = pd.DataFrame.from_dict(results_dic, orient='index',
                       columns=['p value', 'test', 'XGboost rank','XGboost mean decrease in AUC','LR rank','LR mean decrease in AUC'])
training_rank = [((np.where(a==xg_tune.best_estimator_[:-1].get_feature_names_out()[indices]))[0]).item(0) for a in xg_tune.best_estimator_[:-1].get_feature_names_out()]
results_df['XG rank on train via impurity'] = training_rank
results_df['p value'] = np.round(results_df['p value'],4)
results_df['XGboost mean decrease in AUC'] = np.round(results_df['XGboost mean decrease in AUC'],3)
results_df['LR mean decrease in AUC'] = np.round(results_df['LR mean decrease in AUC'],3)
results_df = results_df[['p value', 'test', 'XGboost rank','XGboost mean decrease in AUC','XG rank on train via impurity','LR rank','LR mean decrease in AUC']]
results_df

In [ ]:
#visualizing feature importance via permutation on test- XGBoost VS logistic regression 
x = (np.arange(len(results_df.index)))*4.2  # the label locations

dic4plot = {'XGBoost':results_df.loc[:,"XGboost mean decrease in AUC"].tolist(),'Logistic regression':results_df.loc[:, "LR mean decrease in AUC"].tolist()}
width = 2  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(25, 15),layout='constrained')

for attribute, measurement in dic4plot.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=1,fontsize=18)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Mean decrease in AUC',fontsize=30)
ax.set_xlabel('Features',fontsize=30)
#ax.set_title('Features by algorithm')
ax.set_xticks(x+width/2,['BMI','Platletes','Age','Albumin','ALT','AST','ALP','GGT','HbA1c','HDL-c','LDL-c','Cholesterol','Triglycerides','Female','Male','DM or IFG','Hypertension'],fontsize=25,rotation = 45)
ax.tick_params(axis='y', labelsize=25)
ax.legend(loc='upper left', ncols=2,fontsize = 30)
ax.set_ylim(-0.02, 0.1)

plt.show()

In [ ]:
# XGBoost explainability with shap 
import shap
explainer = shap.TreeExplainer(xg_tune.best_estimator_[2])
shap_interaction_values = explainer.shap_interaction_values(xg_tune.best_estimator_[0:2].transform(df_test))
mean_shap = np.absolute(shap_interaction_values).mean(0)
SHAP_df = pd.DataFrame(mean_shap,index = list(pipe_XG[:-1].get_feature_names_out()), columns = list(pipe_XG[:-1].get_feature_names_out()))

In [ ]:
# The interaction heat map in terms of shap values
SHAP_df_LT = SHAP_df.where(np.tril(np.ones(SHAP_df.shape)).astype(np.bool))
fig, ax = plt.subplots(figsize=(15, 15))
im = ax.imshow(SHAP_df_LT.to_numpy(),norm=None,cmap='Greys')

ax.set_xticks(np.arange(len(list(pipe_XG[:-1].get_feature_names_out()))), labels=['BMI','Platletes','Age','Albumin','ALT','AST','ALP','GGT','HbA1c','HDL-c','LDL-c','Cholesterol','Triglycerides','Female','Male','DM or IFG','Hypertension']
,fontsize=25)
ax.set_yticks(np.arange(len(list(pipe_XG[:-1].get_feature_names_out()))), labels=['BMI','Platletes','Age','Albumin','ALT','AST','ALP','GGT','HbA1c','HDL-c','LDL-c','Cholesterol','Triglycerides','Female','Male','DM or IFG','Hypertension']
,fontsize=25)


plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(len(list(pipe_XG[:-1].get_feature_names_out()))):
    for j in range((len(list(pipe_XG[:-1].get_feature_names_out()))-1),(i-1),-1):
        text = ax.text(i, j,np.round(SHAP_df.to_numpy(),2)[i, j],
                       ha="center", va="center", color="lightslategrey",fontsize=20)
fig.tight_layout()
plt.show()

In [ ]:
# error analysis- identify shap patterns associated with FP
from scipy.stats import shapiro,ttest_ind,mannwhitneyu
shap_interaction_values_FP = explainer.shap_interaction_values(xg_tune.best_estimator_[0:2].transform(df_test.loc[(y_testtt==0) & (xg_tune.best_estimator_.predict_proba(df_test)[:,1]>0.14)]))
shap_interaction_values_TP = explainer.shap_interaction_values(xg_tune.best_estimator_[0:2].transform(df_test.loc[(y_testtt==1) & (xg_tune.best_estimator_.predict_proba(df_test)[:,1]>0.14)]))
mean_shap_FP = pd.DataFrame(shap_interaction_values_FP.mean(0),index = list(pipe_XG[:-1].get_feature_names_out()), columns = list(pipe_XG[:-1].get_feature_names_out()))
mean_shap_TP = pd.DataFrame(shap_interaction_values_TP.mean(0),index = list(pipe_XG[:-1].get_feature_names_out()), columns = list(pipe_XG[:-1].get_feature_names_out()))
# T test-
hypothesis_matirix = np.ones(SHAP_df.shape)

for i in range(len(list(pipe_XG[:-1].get_feature_names_out()))):
    for j in range(len(list(pipe_XG[:-1].get_feature_names_out()))):
        hypothesis_matirix[j,i] = ttest_ind(shap_interaction_values_FP[:,j,i],shap_interaction_values_TP[:,j,i])[1]

SHAP_hypothesis_results = pd.DataFrame(hypothesis_matirix,index = list(pipe_XG[:-1].get_feature_names_out()), columns = list(pipe_XG[:-1].get_feature_names_out()))

In [ ]:
#plot the hypothesis testing for shap of FP vs TN to identify patterns associated with FP
SHAP_hypothesis_LT = SHAP_df.where(np.tril(np.ones(SHAP_df.shape)).astype(np.bool))
fig, ax = plt.subplots(figsize=(20, 15))
im = ax.imshow(SHAP_hypothesis_LT.to_numpy(),norm=None,cmap='OrRd')
ax.set_xticks(np.arange(len(list(pipe_XG[:-1].get_feature_names_out()))), labels=list(pipe_XG[:-1].get_feature_names_out()),fontsize=15)
ax.set_yticks(np.arange(len(list(pipe_XG[:-1].get_feature_names_out()))), labels=list(pipe_XG[:-1].get_feature_names_out()),fontsize=15)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(len(list(pipe_XG[:-1].get_feature_names_out()))):
    for j in range((len(list(pipe_XG[:-1].get_feature_names_out()))-1),(i-1),-1):
        if (SHAP_hypothesis_results.to_numpy()[i,j]<0.001):
            text = ax.text(i, j,np.round(SHAP_hypothesis_results.to_numpy(),5)[i, j],
                       ha="center", va="center", color="black",fontsize=15)
        else:
            text = ax.text(i, j,'NS',
                       ha="center", va="center", color="black",fontsize=15)
fig.tight_layout()
plt.show()

In [ ]:
# standard deviation for shap values FP or TP
np.std(shap_interaction_values_TP[:,1,1]) # choose your fature shap in dimensions - TP
np.std(shap_interaction_values_FP[:,1,1]) # choose your fature shap in dimensions - FP

In [ ]:
# examine feature value (not shap) of FP and TP
df_test.loc[(y_testtt==1) & (xg_tune.best_estimator_.predict_proba(df_test)[:,1]>0.14)].describe()
df_test.loc[(y_testtt==0) & (xg_tune.best_estimator_.predict_proba(df_test)[:,1]>0.14)].describe()

# perform univariate hypothesis testing betweeen FP and TP for each feature as described 
mannwhitneyu(df_test.loc[(y_testtt==1) & (xg_tune.best_estimator_.predict_proba(df_test)[:,1]>0.14)]['ALT'],
             df_test.loc[(y_testtt==0) & (xg_tune.best_estimator_.predict_proba(df_test)[:,1]>0.14)]['ALT'],nan_policy='omit')[1]

In [ ]:
# waterfall plot with shap

shap_values = explainer(xg_tune.best_estimator_[0:2].transform(df_test))
shap.plots.waterfall(shap_values[12],max_display=15,show=False)# 12 false positive
fig, ax = plt.gcf(), plt.gca()


ax.set_yticks(np.arange(0,15,1),labels=['TG (116) and ALP (114)','Female=Yes','LDL-c=130','Cholesterol=209','ALT=33','HDL-c=55','Hypertension=Yes','DM or IFG=No','Age=63','HbA1c=5.5','AST=31','Platletes=256','GGT=31','BMI=39','Albumin=3.6'],fontsize=25)


fig.set_figheight(15)
fig.set_figwidth(20)
plt.show()

In [ ]:
# describe the SHAP interaction of GGT and ALT
fig, ax = plt.subplots(figsize=(10, 10))
points=ax.scatter(x=df_test.loc[(y_testtt==0) & (xg_tune.best_estimator_.predict_proba(df_test)[:,1]>0.14)].ALT,y=shap_interaction_values_FP[:,4,7],c=df_test.loc[(y_testtt==0) & (xg_tune.best_estimator_.predict_proba(df_test)[:,1]>0.14)].GGT,cmap='viridis',vmin=20,vmax=150)
bar=fig.colorbar(points,ax=ax, pad=0.08, shrink=0.6, aspect=10)
bar.ax.set_title("GGT (U/L)",fontsize=15)
bar.ax.tick_params(labelsize=15)
ax.set_xlabel('ALT (U/L)',fontsize=15)
ax.set_ylabel('SHAP interaction value',fontsize=15)
plt.show()

In [ ]:
### XGBoost calibration on the validation sample
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import CalibrationDisplay
calibrated_xg = CalibratedClassifierCV(xg_tune.best_estimator_, cv="prefit",method="sigmoid")
calibrated_xg.fit(df_test, y_testtt)
disp = CalibrationDisplay.from_estimator(xg_tune.best_estimator_, df_test, y_testtt,name='XGBoost')

plt.xlabel('Mean predicted probability by XGBoost')
plt.ylabel('Fraction of patients with advanced fibrosis')
plt.show()